In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

C:\Users\tomas\ana.spy\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Importing Cleaned Data

In [2]:
urlTransfer = 'https://raw.githubusercontent.com/edb-313/Football_players_trasfer_value/main/Transfers/Transfers_cleaned.csv'
urlStats = 'https://raw.githubusercontent.com/edb-313/Football_players_trasfer_value/main/StatsData/StatsData_Cleaned.csv'
urlContractDate = 'https://raw.githubusercontent.com/edb-313/Football_players_trasfer_value/main/ContractDate/ContractDates_cleaned.csv'

transfers_df = pd.read_csv(urlTransfer)
stats_df = pd.read_csv(urlStats)
contractDate_df = pd.read_csv(urlContractDate)

In [3]:
transfers_df.head()

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year
0,Ángel Di María,22.0,Right Winger,Real Madrid,Summer,33000000.0,2010
1,Ramires,23.0,Central Midfield,Chelsea,Summer,22000000.0,2010
2,Rafik Halliche,23.0,Centre-Back,Fulham,Summer,2500000.0,2010
3,Ariza Makukula,29.0,Centre-Forward,Manisaspor,Summer,1700000.0,2010
4,David Luiz,23.0,Centre-Back,Chelsea,Winter,25000000.0,2010


In [4]:
stats_df.head()

,Player,Nation,Pos,Age,cum_MP,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,Season
0,Quim,pt POR,GK,31,30.0,2639.0,0.0,0.0,0.0,1.0,1.0,0.00,0.00,0.00,2007
1,Rui Costa,pt POR,MF,35,29.0,2382.0,5.0,5.0,0.0,6.0,0.0,0.19,0.19,0.38,2007
2,Kostas Katsouranis,gr GRE,DFMF,28,27.0,2320.0,2.0,1.0,0.0,9.0,0.0,0.08,0.04,0.12,2007
3,Léo,br BRA,DF,32,27.0,2323.0,0.0,3.0,0.0,2.0,0.0,0.00,0.12,0.12,2007
4,Óscar Cardozo,py PAR,FW,24,29.0,2283.0,13.0,1.0,3.0,2.0,0.0,0.51,0.04,0.55,2007


In [5]:
transfers_df[transfers_df['player_name']=='Bernardo Silva']

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year
31,Bernardo Silva,20.0,Attacking Midfield,Monaco,Winter,15750000.0,2014


In [6]:
stats_df[stats_df['Player']=='Bernardo Silva']

,Player,Nation,Pos,Age,cum_MP,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,Season
189,Bernardo Silva,pt POR,FWMF,18,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2013


### Merging transfers_df and stats_df

In [7]:
#If transfer was done in summer, we have to use stats from previous year
transfers_df['stats_year_summer'] = transfers_df['year']-1

In [8]:
# create a mask to filter summer transfers
summer_mask = transfers_df['transfer_period'] == 'Summer'

# create a mask to filter winter transfers
winter_mask = transfers_df['transfer_period'] == 'Winter'

# perform join for summer transfers
summer_join = pd.merge(transfers_df[summer_mask], stats_df, left_on=['player_name', 'stats_year_summer'], right_on=['Player', 'Season'], how='inner')

# perform join for winter transfers
winter_join = pd.merge(transfers_df[winter_mask], stats_df, left_on=['player_name', 'year'], right_on=['Player', 'Season'], how='inner')

# concatenate the results
merged_df = pd.concat([summer_join, winter_join], ignore_index=True)

In [9]:
merged_df.head()

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year,stats_year_summer,Player,Nation,...,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,Season
0,Ángel Di María,22.0,Right Winger,Real Madrid,Summer,33000000.0,2010,2009,Ángel Di María,ar ARG,...,2141.138045,4.651992,9.944212,0.0,8.270653,0.891186,0.192942,0.401788,0.594730,2009
1,Ramires,23.0,Central Midfield,Chelsea,Summer,22000000.0,2010,2009,Ramires,br BRA,...,1945.000000,4.000000,3.000000,0.0,2.000000,0.000000,0.190000,0.140000,0.320000,2009
2,Fábio Coentrão,23.0,Left-Back,Real Madrid,Summer,30000000.0,2011,2010,Fábio Coentrão,pt POR,...,2018.823073,1.780261,3.257877,0.0,7.210056,1.780261,0.080112,0.145091,0.225203,2010
3,Roberto,25.0,Goalkeeper,Real Zaragoza,Summer,8600000.0,2011,2010,Roberto,es ESP,...,2227.000000,0.000000,0.000000,0.0,2.000000,0.000000,0.000000,0.000000,0.000000,2010
4,Axel Witsel,23.0,Defensive Midfield,Zenit S-Pb,Summer,40000000.0,2012,2011,Axel Witsel,be BEL,...,2281.000000,1.000000,3.000000,1.0,5.000000,0.000000,0.040000,0.120000,0.160000,2011


For some players, the name isn't exactly equal in transfers_df and stats_df. We will now use fuzzywuzzy to find the best match for each transfer_df['player_name'], and then join stats_df and transfer_df based on the best match

In [10]:
# create a list of player names in 'stats_df'
stats_player_names = stats_df['Player'].unique()

# create a function to find the best match for a given player name
def find_best_match(player_name):
    best_ratio = -1
    best_match = None
    for stats_player_name in stats_player_names:
        ratio = fuzz.token_sort_ratio(player_name, stats_player_name)
        if ratio > best_ratio:
            best_ratio = ratio
            best_match = stats_player_name
    return best_match

# create a column in 'transfers_df' with the best matching player name from 'stats_df'
transfers_df['best_match'] = transfers_df['player_name'].apply(find_best_match)

# perform join for summer transfers
summer_join = pd.merge(transfers_df[summer_mask], stats_df, left_on=['best_match', 'stats_year_summer'], right_on=['Player', 'Season'], how='inner')

# perform join for winter transfers
winter_join = pd.merge(transfers_df[winter_mask], stats_df, left_on=['best_match', 'year'], right_on=['Player', 'Season'], how='inner')

# concatenate the results
merged_fuzzy_df= pd.concat([summer_join, winter_join], ignore_index=True)

Now that we have merged_fuzzy_df, let's check what additional joins we were able to make to do in respect to the initial merge

In [11]:
#remove best_match column as it is present only on merged_fuzzy_df and not in merged_df
merged_fuzzy_df.drop(['best_match'],axis=1,inplace=True)

In [12]:
#Concatenate the two merged dfs
double_merged_df = pd.concat([merged_df, merged_fuzzy_df], ignore_index=True)

#Drop the duplicates to remain only with the rows present only in merged_fuzzy_df
double_merged_df.drop_duplicates(keep=False, inplace=True)

In [13]:
#Display double_merged_df
double_merged_df

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year,stats_year_summer,Player,Nation,...,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,Season
47,Daniel Wass,23.0,Central Midfield,Évian,Summer,1200000.0,2012,2011,Axel Witsel,be BEL,...,2281.000000,1.000000,3.0,1.000000,5.000000,0.00000,0.040000,0.12,0.160000,2011
49,Émerson Conceição,26.0,Left-Back,Trabzonspor,Summer,1600000.0,2012,2011,Emerson,br BRA,...,2093.000000,0.000000,1.0,0.000000,6.000000,1.00000,0.000000,0.04,0.040000,2011
52,Lazar Markovic,20.0,Right Winger,Liverpool,Summer,25000000.0,2014,2013,Lazar Marković,rs SRB,...,1659.000000,5.000000,5.0,0.000000,2.000000,0.00000,0.270000,0.27,0.540000,2013
56,Stefan Mitrović,24.0,Centre-Back,SC Freiburg,Summer,1500000.0,2014,2013,Nemanja Matić,rs SRB,...,1326.325818,2.087247,0.0,0.000000,6.196061,0.09803,0.146520,0.00,0.146520,2013
62,Mehdi Carcela-González,27.0,Right Winger,Granada CF,Summer,4000000.0,2016,2015,Mehdi Carcela,ma MAR,...,822.000000,2.000000,3.0,0.000000,1.000000,0.00000,0.220000,0.33,0.550000,2015
66,Konstantinos Mitroglou,29.0,Centre-Forward,Marseille,Summer,15000000.0,2017,2016,Kostas Mitroglou,gr GRE,...,2231.738739,16.396396,3.0,0.099099,2.297297,0.00000,0.657838,0.12,0.777838,2016


Using fuzzy we were able to find additional matches shown in rows 1,2,4,5. Let's add these rows to our first merged dataframe.

In [14]:
#Selecting only the rows fuzzy was able to match correctly
double_merged_df = double_merged_df.iloc[[1,2,4,5]]

In [15]:
#Adding them to merged_df
merged_df = pd.concat([merged_df, double_merged_df], ignore_index=True)

In [16]:
#Cleaning merged_df of unneccesary columns
merged_df.drop(['Player','Nation', 'Pos', 'Age','Season','stats_year_summer'],axis=1,inplace=True)

### Merging with ContractDate_df


In [17]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   player_name         46 non-null     object 
 1   age                 46 non-null     float64
 2   position            46 non-null     object 
 3   club_involved_name  46 non-null     object 
 4   transfer_period     46 non-null     object 
 5   fee_cleaned         46 non-null     float64
 6   year                46 non-null     int64  
 7   cum_MP              46 non-null     float64
 8   cum_Min             46 non-null     float64
 9   cum_Gls             46 non-null     float64
 10  cum_Ast             46 non-null     float64
 11  cum_PK              46 non-null     float64
 12  cum_CrdY            46 non-null     float64
 13  cum_CrdR            46 non-null     float64
 14  cum_GlsxMin         46 non-null     float64
 15  cum_AstxMin         46 non-null     float64
 16  cum_G+AxMi

In [18]:
contractDate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 276 entries, 0 to 275
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   player_name     276 non-null    object 
 1   tf_season       276 non-null    int64  
 2   days_remaining  276 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 6.6+ KB


In [19]:
#performing the merge of contractDate_df and merged_df on player_name and year
full_df = pd.merge(merged_df, contractDate_df, left_on=['player_name','year'], right_on=['player_name','tf_season'], how='left')

In [20]:
#dropping unneccesary columns of full_df
full_df.drop('tf_season',axis=1,inplace=True)

### Manually inputting missing values in column 'days_remaining'

Filtering out values that are NaN

In [21]:
missing_values = full_df[full_df['days_remaining'].isnull()]

In [22]:
missing_values

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year,cum_MP,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,days_remaining
0,Ángel Di María,22.0,Right Winger,Real Madrid,Summer,33000000.0,2010,25.803939,2141.138045,4.651992,9.944212,0.0,8.270653,0.891186,0.192942,0.401788,0.594730,NaN
1,Ramires,23.0,Central Midfield,Chelsea,Summer,22000000.0,2010,26.000000,1945.000000,4.000000,3.000000,0.0,2.000000,0.000000,0.190000,0.140000,0.320000,NaN
3,Roberto,25.0,Goalkeeper,Real Zaragoza,Summer,8600000.0,2011,25.000000,2227.000000,0.000000,0.000000,0.0,2.000000,0.000000,0.000000,0.000000,0.000000,NaN
4,Axel Witsel,23.0,Defensive Midfield,Zenit S-Pb,Summer,40000000.0,2012,29.000000,2281.000000,1.000000,3.000000,1.0,5.000000,0.000000,0.040000,0.120000,0.160000,NaN
5,Javi García,25.0,Defensive Midfield,Man City,Summer,20200000.0,2012,22.239194,1740.948044,1.119597,0.119597,0.0,8.901970,0.098030,0.054676,0.004784,0.059460,NaN
9,Ezequiel Garay,27.0,Centre-Back,Zenit S-Pb,Summer,6000000.0,2014,26.967650,2420.634525,5.466714,0.000000,0.0,2.901970,0.000000,0.200845,0.000000,0.200845,NaN
15,Nicolás Gaitán,28.0,Attacking Midfield,Atlético Madrid,Summer,25000000.0,2016,25.204215,2045.895958,3.998728,13.709884,0.0,2.446500,0.000000,0.178973,0.605309,0.783411,NaN
33,Bruno César,24.0,Central Midfield,Al-Ahli SFC,Winter,5000000.0,2012,7.981982,364.216216,0.891892,0.198198,0.0,2.396396,0.000000,0.048559,0.010901,0.059459,NaN
36,Franco Jara,26.0,Centre-Forward,Olympiacos,Winter,1500000.0,2014,1.784471,94.580135,0.000782,0.001315,0.0,0.890405,0.000000,0.000069,0.001196,0.001265,NaN
40,Enzo Fernández,22.0,Central Midfield,Chelsea,Winter,121000000.0,2022,22.000000,1809.000000,1.000000,6.000000,0.0,6.000000,0.000000,0.050000,0.300000,0.350000,NaN


Collecting and inputting values

In [23]:
days_remaining_list = [1825, 1782, 365, 1761, 760, 760 , 1090, 1441, 157, 1611, 1364]

# Inserting values manually in every row that has NaN
for i, row in missing_values.iterrows():
    missing_values.at[i, 'days_remaining'] = days_remaining_list[i % len(days_remaining_list)]

In [24]:
missing_values

,player_name,age,position,club_involved_name,transfer_period,fee_cleaned,year,cum_MP,cum_Min,cum_Gls,cum_Ast,cum_PK,cum_CrdY,cum_CrdR,cum_GlsxMin,cum_AstxMin,cum_G+AxMin,days_remaining
0,Ángel Di María,22.0,Right Winger,Real Madrid,Summer,33000000.0,2010,25.803939,2141.138045,4.651992,9.944212,0.0,8.270653,0.891186,0.192942,0.401788,0.594730,1825.0
1,Ramires,23.0,Central Midfield,Chelsea,Summer,22000000.0,2010,26.000000,1945.000000,4.000000,3.000000,0.0,2.000000,0.000000,0.190000,0.140000,0.320000,1782.0
3,Roberto,25.0,Goalkeeper,Real Zaragoza,Summer,8600000.0,2011,25.000000,2227.000000,0.000000,0.000000,0.0,2.000000,0.000000,0.000000,0.000000,0.000000,1761.0
4,Axel Witsel,23.0,Defensive Midfield,Zenit S-Pb,Summer,40000000.0,2012,29.000000,2281.000000,1.000000,3.000000,1.0,5.000000,0.000000,0.040000,0.120000,0.160000,760.0
5,Javi García,25.0,Defensive Midfield,Man City,Summer,20200000.0,2012,22.239194,1740.948044,1.119597,0.119597,0.0,8.901970,0.098030,0.054676,0.004784,0.059460,760.0
9,Ezequiel Garay,27.0,Centre-Back,Zenit S-Pb,Summer,6000000.0,2014,26.967650,2420.634525,5.466714,0.000000,0.0,2.901970,0.000000,0.200845,0.000000,0.200845,1611.0
15,Nicolás Gaitán,28.0,Attacking Midfield,Atlético Madrid,Summer,25000000.0,2016,25.204215,2045.895958,3.998728,13.709884,0.0,2.446500,0.000000,0.178973,0.605309,0.783411,760.0
33,Bruno César,24.0,Central Midfield,Al-Ahli SFC,Winter,5000000.0,2012,7.981982,364.216216,0.891892,0.198198,0.0,2.396396,0.000000,0.048559,0.010901,0.059459,1825.0
36,Franco Jara,26.0,Centre-Forward,Olympiacos,Winter,1500000.0,2014,1.784471,94.580135,0.000782,0.001315,0.0,0.890405,0.000000,0.000069,0.001196,0.001265,1761.0
40,Enzo Fernández,22.0,Central Midfield,Chelsea,Winter,121000000.0,2022,22.000000,1809.000000,1.000000,6.000000,0.0,6.000000,0.000000,0.050000,0.300000,0.350000,1441.0


Inputtining manually collected values to main df 

In [25]:
# left join on 'player_name' column
full_df = full_df.merge(missing_values[['player_name', 'days_remaining']], on='player_name', how='left', suffixes=('', '_missing'))

# replace missing values in 'days_remaining' column with corresponding values from 'days_remaining_missing'
full_df['days_remaining'] = full_df['days_remaining_missing'].combine_first(full_df['days_remaining'])
full_df.drop(columns=['days_remaining_missing'], inplace=True)

In [26]:
full_df.to_csv("C:/Users/tomas/OneDrive/Dokumentai/GitHub/Football_players_trasfer_value/joined_final.csv",index=False)